# Import Modules

In [1]:
import requests

# Météo

## Extract
From website [https://www.meteociel.fr/climatologie](https://www.meteociel.fr/climatologie)

In [ ]:
#Extract scrap from website
#https://www.meteociel.fr/climatologie/obs_villes.php?code2=75107005&mois=12&annee=2025&sn=0


# Load in SQL Lite

## Transform

## Load

# Vélo

## Extract
From API [https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/](https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/api/?disjunctive.id_compteur&disjunctive.nom_compteur&disjunctive.id&disjunctive.name)


## Transform

## Load

In [3]:
# E T L Vélo